# EXPLORATION 1. 인공지능과 가위바위보 하기

---

##  Dataset

### Data 만들기

teachable machine 사이트에서 데이터를 만들어 냅니다.

[teachablemachine](https://teachablemachine.withgoogle.com/)

웹캠을 이용하여, 가위,바위,보 사진을 만들어냅니다.

### Resize 하기

In [1]:
from PIL import Image
import os, glob
# 가위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/scissor"
print("이미지 디렉토리 경로: ", image_dir_path)

images=glob.glob(image_dir_path + "/*.jpg")  

# 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
target_size=(28,28)
for img in images:
    old_img=Image.open(img)
    new_img=old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")

print("가위 이미지 resize 완료!")


# 바위 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
# [[YOUR CODE]]
rock_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/rock"
rock_images = glob.glob(rock_dir_path + "/*.jpg")

# 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
# [[YOUR CODE]]
target_size=(28,28)
for img in rock_images:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img, "JPEG")


print("바위 이미지 resize 완료!")


# 보 이미지가 저장된 디렉토리 아래의 모든 jpg 파일을 읽어들여서
# [[YOUR CODE]]
paper_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/paper"
paper_images = glob.glob(paper_dir_path + "/*.jpg")

# 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
# [[YOUR CODE]]
target_size = (28,28)
for img in paper_images:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img, "JPEG")
    
print("보 이미지 resize 완료!")

이미지 디렉토리 경로:  /home/aiffel0046/aiffel/rock_scissor_paper/scissor
가위 이미지 resize 완료!
바위 이미지 resize 완료!
보 이미지 resize 완료!


### Data 불러오기

In [15]:
import numpy as np
def load_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=6600   # 가위바위보 이미지 개수 총합에 주의하세요.
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color, dtype=np.int32).reshape(number_of_data, img_size, img_size, color)
    labels=np.zeros(number_of_data, dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file), dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는",idx,"입니다.")
    return imgs, labels

image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 6600 입니다.
x_train shape: (6600, 28, 28, 3)
y_train shape: (6600,)


## Model

### model 설계하기

In [16]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# model을 직접 만들어 보세요.
# Hint! model의 입력/출력부에 특히 유의해 주세요. 가위바위보 데이터셋은 MNIST 데이터셋과 어떤 점이 달라졌나요?
# [[YOUR CODE]]
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(128, (3,3), activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))


model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 64)        1792      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 128)       73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                204864    
_________________________________________________________________
dense_5 (Dense)              (None, 3)                

### 모델 학습하기

In [17]:
# model을 학습시키는 코드를 직접 작성해 보세요.
# Hint! model.compile()과 model.fit()을 사용해 봅시다.
# [[YOUR CODE]]
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# 모델 훈련
model.fit(x_train_norm, y_train , epochs=10)

Epoch 1/10
207/207 [==============================] - 0s 2ms/step - loss: 0.8252 - accuracy: 0.5905
Epoch 2/10
207/207 [==============================] - 0s 2ms/step - loss: 0.3519 - accuracy: 0.8618
Epoch 3/10
207/207 [==============================] - 0s 2ms/step - loss: 0.2048 - accuracy: 0.9264
Epoch 4/10
207/207 [==============================] - 0s 2ms/step - loss: 0.1335 - accuracy: 0.9574
Epoch 5/10
207/207 [==============================] - 0s 2ms/step - loss: 0.0831 - accuracy: 0.9764
Epoch 6/10
207/207 [==============================] - 0s 2ms/step - loss: 0.0585 - accuracy: 0.9836
Epoch 7/10
207/207 [==============================] - 0s 2ms/step - loss: 0.0326 - accuracy: 0.9929
Epoch 8/10
207/207 [==============================] - 0s 2ms/step - loss: 0.0183 - accuracy: 0.9964
Epoch 9/10
207/207 [==============================] - 0s 2ms/step - loss: 0.0168 - accuracy: 0.9968
Epoch 10/10
207/207 [==============================] - 0s 2ms/step - loss: 0.0072 - accuracy: 0.9998

### Test data 불러오기

In [18]:
# x_test, y_test를 만드는 방법은 x_train, y_train을 만드는 방법과 아주 유사합니다.
# [[YOUR CODE]]
def load_test(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=900   # 가위바위보 이미지 개수 총합에 주의하세요.
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color, dtype=np.int32).reshape(number_of_data, img_size, img_size, color)
    labels=np.zeros(number_of_data, dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file), dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    return imgs, labels

In [19]:
test_image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/test"
print(test_image_dir_path)

# scissor
t_s_images=glob.glob(test_image_dir_path + "/scissor/*.jpg")  
target_size=(28,28)
for img in t_s_images:
    old_img=Image.open(img)
    new_img=old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")

print("가위 이미지 resize 완료!")

# rock
t_r_images=glob.glob(test_image_dir_path + "/rock/*.jpg")  
target_size=(28,28)
for img in t_r_images:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img, "JPEG")
 
print("바위 이미지 resize 완료!")

# paper
t_p_images=glob.glob(test_image_dir_path + "/paper/*.jpg")  
target_size=(28,28)
for img in t_p_images:
    old_img = Image.open(img)
    new_img = old_img.resize(target_size, Image.ANTIALIAS)
    new_img.save(img, "JPEG")
 
print("바위 이미지 resize 완료!")


(x_test, y_test)=load_test(test_image_dir_path)
x_test_norm = x_test/255.0

print(x_test.shape)

/home/aiffel0046/aiffel/rock_scissor_paper/test
가위 이미지 resize 완료!
바위 이미지 resize 완료!
바위 이미지 resize 완료!
(900, 28, 28, 3)


### Model Test 하기

In [20]:
# model을 학습시키는 코드를 직접 작성해 보세요.
# Hint! model.evaluate()을 사용해 봅시다.
# [[YOUR CODE]]
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

29/29 - 0s - loss: 0.2320 - accuracy: 0.9278
test_loss: 0.23197443783283234 
test_accuracy: 0.9277777671813965


## Report

### 초기 데이터셋

Train - 300 장

scissor : 100 장  
rock    : 100 장  
paper   : 100 장   
    
Test - 300 장

scissor : 100 장  
rock    : 100 장  
paper   : 100 장  

처음에 이 데이터셋을 학습하고 테스트 했을 때는 성능이 좋지 않았습니다. 제일 높게 나온 test acc 는 약 0.44정도 였습니다.

### 데이터셋 추가

Train - 6600 장

scissor : 2200 장  
rock    : 2200 장  
paper   : 2200 장   
    
Test - 900 장

scissor : 300 장  
rock    : 300 장  
paper   : 300 장  

팀원들의 데이터를 추가하여, 학습을 진행하였습니다.
결과적으로 test acc가 0.92 정도로 많은 향상을 보였습니다.